In [2]:
def encode(plain_text, private_key):
    assert plain_text in {0, 1} and private_key in {0, 1}
    return plain_text ^ private_key

In [3]:
def decode(cipher_text, private_key):
    assert cipher_text in {0, 1} and private_key in {0, 1}
    return cipher_text ^ private_key

In [4]:
from itertools import product

In [5]:
?product

Init signature: product(self, /, *args, **kwargs)
Docstring:     
product(*iterables, repeat=1) --> product object

Cartesian product of input iterables.  Equivalent to nested for-loops.

For example, product(A, B) returns the same as:  ((x,y) for x in A for y in B).
The leftmost iterators are in the outermost for-loop, so the output tuples
cycle in a manner similar to an odometer (with the rightmost element changing
on every iteration).

To compute the product of an iterable with itself, specify the number
of repetitions with the optional repeat keyword argument. For example,
product(A, repeat=4) means the same as product(A, A, A, A).

product('ab', range(3)) --> ('a',0) ('a',1) ('a',2) ('b',0) ('b',1) ('b',2)
product((0,1), (0,1), (0,1)) --> (0,0,0) (0,0,1) (0,1,0) (0,1,1) (1,0,0) ...
Type:           type
Subclasses:     

In [6]:
for plaintext, private_key in product({0, 1}, repeat=2):
    ciphertext = encode(plaintext, private_key)
    print(f'key: {private_key}, '
          f'plaintext: {plaintext}, '
          f'ciphertext: {ciphertext}, '
          f'decoded: {decode(ciphertext, private_key)}')

key: 0, plaintext: 0, ciphertext: 0, decoded: 0
key: 1, plaintext: 0, ciphertext: 1, decoded: 0
key: 0, plaintext: 1, ciphertext: 1, decoded: 1
key: 1, plaintext: 1, ciphertext: 0, decoded: 1


In [1]:
def to_hex(plain_text):
    hex_codes = []
    for symbol in plain_text:
        hex_code = hex(ord(symbol)).replace('0x', '')
        if len(hex_code) == 1:
            hex_code = '0' + hex_code
        hex_codes.append(hex_code)
    return ''.join(hex_codes)

In [2]:
def to_str(hex_code):
    if hex_code:
        return chr(int(hex_code[:2], base=16)) + to_str(hex_code[2:])
    return ''

In [3]:
message = 'Hello World'
print(f'hex of {message} is: {to_hex(message)}')

hex of Hello World is: 48656c6c6f20576f726c64


In [4]:
code = '736f6d65206d657373616765'
print(f'str of {code} is: {to_str(code)}')

str of 736f6d65206d657373616765 is: some message


In [5]:
def bitwise_xor(first_text, second_text):
    assert len(first_text) == len(second_text)
    return ''.join(format(int(s1, 16) ^ int(s2, 16), '01x')
                   for s1, s2 in zip(first_text, second_text))

In [6]:
message = 'secret message'
key =     'my secret keys'
print(f'hex of {message} is: {to_hex(message)}')
print(to_hex(key))

hex of secret message is: 736563726574206d657373616765
6d7920736563726574206b657973


In [7]:
ciphertext = bitwise_xor(to_hex(message), to_hex(key))
print('ciphertext:', ciphertext)

ciphertext: 1e1c430100175208115318041e16


In [8]:
recovered_message = to_str(bitwise_xor(ciphertext, to_hex(key)))
print('recovered message:', recovered_message)

recovered message: secret message


In [9]:
message1 = 'steal the secret'
message2 = 'the boy the girl'
key = 'supersecretverys'

In [10]:
ciphertext1 = bitwise_xor(to_hex(message1), to_hex(key))
ciphertext2 = bitwise_xor(to_hex(message2), to_hex(key))

In [11]:
xor_ciphertexts = bitwise_xor(ciphertext1, ciphertext2)
xor_messages = bitwise_xor(to_hex(message1), to_hex(message2))

In [12]:
print(xor_ciphertexts)
print(xor_messages)

071c00410e4f0d4811481645041b1718
071c00410e4f0d4811481645041b1718


In [13]:
if xor_ciphertexts == xor_messages:
    print('Xor of the ciphertexts is the same as xor of messages')
else:
    print('Xor of the ciphertexts differs from the xor of messages')

Xor of the ciphertexts is the same as xor of messages


In [14]:
def try_guessing_substring(substring, message_length, xor_messages):
    good_guesses = []
    for pos in range(message_length - len(substring) + 1):
        guess = to_hex(chr(0) * pos + substring +
                       chr(0) * (message_length - len(substring) - pos))
        other_message_part = to_str(
            bitwise_xor(guess, xor_messages)
        )[pos:pos + len(substring)]
        good_guess = True
        for i in range(len(other_message_part)):
            if not other_message_part[i].isalpha() and \
                    not other_message_part[i].isspace():
                good_guess = False
                break
        if good_guess:
            good_guesses.append((guess, pos, other_message_part))

    print('Good guesses:')
    for guess in good_guesses:
        print(f'position: {guess[1]}, '
              f'one message part: \"{substring}\", '
              f'another message part: \"{guess[2]}\"')

In [15]:
try_guessing_substring(' the ', len(message1), xor_messages)

Good guesses:
position: 5, one message part: " the ", another message part: "oy th"
position: 7, one message part: " the ", another message part: "he se"


In [16]:
try_guessing_substring('oy the ', len(message1), xor_messages)

Good guesses:
position: 5, one message part: "oy the ", another message part: " the se"
